# Evaluation Methods for Topic Models

A first recap of inference in statistical models, and how to use parameters -- or distributions over parameters -- inferred from data $\mathcal{X}$ to estimate the probability of new observations $x_*$.

There are two things we look at

1. Predictive Likelihood
2. Document Completion



# Predictive Likelihood

We consider models whose "global" parameters are $\Phi$ and whose observation-local parameters -- if they exist -- are $\theta_d$ for each document $d$. We also consider document-local indicators $z_d$ 

We want to predict the likelihood of a new observation $x_*$ according to our inferred parameters.

## ML and MAP 

For maximum likelihood, and MAP, we learn a _value_ $\Phi$ (not a distribution) and then use that value, so we have

$$
\begin{align*}
p(x_*|\Phi) = \int_{\theta_*} \sum_{z_*} p(x_*|z_*, \Phi)p(z|\theta_*)p(\theta_*|\alpha)
\end{align*}
$$

## Bayesian Methods

For Bayesian methods we want to fully capture our uncertainty, and so _avoid_ doing point estimates.

Hence our predictive likelihood is

$$
\begin{align*}
p(x_*|\mathcal{X}) = \int_\Phi \int_{\theta_*} \sum_{z_*}
p(x_*|\Phi, z_*)p(z_*|\theta_*)p(\theta_*|\alpha)p(\Phi, \Theta, \alpha|\mathcal{X})
\end{align*}
$$

We treat the hyper-parameters $\alpha$ as parameters, so in fact we're using $p(x_*|\mathcal{X}; \alpha)$

There are three approaches to this. 

### Point Estimates
The first is just use point estimates, i.e. let

$$
\begin{align*}
\hat{\Phi} & = \mathbb{E}_{\Phi|\mathcal{X}}\left[\Phi\right]
\end{align*}
$$

And substitute in, at which point

$$
\begin{align}
p(x_*|\mathcal{X}, \Phi) & = \int_\Phi \int_{\theta_*} \sum_{z_*} p(x_*|z_*, \Phi)p(z|\theta_*)p(\theta_*|\alpha)p(\Phi|\mathcal{X})\\
 & \approx \int_\Phi \int_{\theta_*} \sum_{z_*} p(x_*|z_*, \hat{\Phi})p(z|\theta_*)p(\theta_*|\alpha)p(\Phi|\mathcal{X})\\
 & = \int_{\theta_*} \sum_{z_*} p(x_*|z_*, \hat{\Phi})p(z|\theta_*)p(\theta_*|\alpha) & & \mathbb{E}\left[{c}\right] = c \text{ for any constant }c
\end{align}
$$

### Sampling

We can approximate

$$
\begin{align}
p(x_*|\mathcal{X}, \Phi) & = \int_\Phi \int_{\theta_*} \sum_{z_*} p(x_*|z_*, \Phi)p(z|\theta_*)p(\theta_*|\alpha)p(\Phi|\mathcal{X})\\
 & \approx \frac{1}{S} \sum_s \int_{\theta_*} \sum_{z_*} p(x_*|z_*, \hat{\Phi}_s)p(z|\theta_*)p(\theta_*|\alpha) & & \hat{\Phi}_s \sim p(\Phi|\mathcal{X})
\end{align}
$$

For this to work we need a sampling distribution $p(\Phi|\mathcal{X})$. 

Frequently you won't have this, but will instead have an approximation $q(\Phi)$, which you can compare against $p(\Phi|\mathcal{X})$ to get samples $\Phi_s$. Metropolis methods, particularly, Gibbs sampling, can be used for this purpose. In general if you have used Gibbs sampling for inference, you will have a source of samples that already define the posterior and can be used for this purpose.

Since we are only interested in the evaluating the expectation $\mathbb{E}_{\Phi|\mathcal{X}}\left[p(x_*|\Phi)\right]$ we might be tempted to use importance sampling, in which samples from $\hat{\Phi}_s \sim q(\Phi)$ are weighted by $p(\hat{\Phi}|\mathcal{X})\text{ / }q(\hat{\Phi})$. However note that for text in particular the dimensionality of $\Phi$ is very large (typically more than $K \times 10000$ for $K$ topics), and importance sampling is notoriously bad in high dimensions with potentially infinite variance (FIXME why). 

The use of importance sampling is particularly tempting when $q(\Phi)$ is a variational posterior, however since this degenerate behaviour can in particular be triggered by a mismatch in the tails of distributions, and since variational methods are known (cite Bishop) to often understate variance due to their mode-seeking behaviour, importance sampling is peculiarly unsuited to estimation of expections by means of variational proposal distributions.

### Deterministic, Variational Approximations



Assume we have an approximation for our posterior, which is $q(\Phi) \approx p(\Phi|\mathcal{X})$. Typically this approximation is chosen to simplify analytical inference, and so often factorises in useful ways.

In that case we obtain

$$
\begin{align}
p(x_*|\mathcal{X}, \Phi) & = \int_\Phi \int_{\theta_*} \sum_{z_*} p(x_*|z_*, \Phi)p(z|\theta_*)p(\theta_*|\alpha)p(\Phi|\mathcal{X})\\
 & \approx \int_\Phi \int_{\theta_*} \sum_{z_*} p(x_*|z_*, \hat{\Phi}_s)p(z|\theta_*)p(\theta_*|\alpha) q(\Phi)
\end{align}
$$

and then analytically solve the integral

# Two Particular Issues that Affect Topic Model Distributions

## Observed Word Orders and the Multinomial Coefficient

Consider the word 'MISSISSIPPI'. The probability of the first letter -- assuming just 26 characters in the (Latin) alphabet -- is 1/26, and so on for the rest of the letters, so it's essentially

$$\text{Pr}(X = {\tt{'MISSISSIPPI'}}) = \prod_i \frac{1}{26}$$

If letters had differing probabilities $\phi_1, \ldots \phi_{26}$ then we would have

$$\text{Pr}(X = {\tt{'MISSISSIPPI'}}) = \prod_i \prod_l \phi_{l}^{x_{il}}$$

where $x_{il}$ is a 1-of-26 indicator vector which indicates which letter $l$ is observed for which character position $i$. For example, the third letter is 'S', which is the 19th letter of the alphabet, so $x_{3,19} = 1$ and $x_{3,l \neq 19} = 0$

Assume instead that we represent this is a bag of letters vector, i.e. counts of each individual letters. We will have $\{{\tt{'M'}}:1, {\tt{'I'}}:4, {\tt{'S'}}: 4, {\tt{'P'}}: 2 \}$ and zero for everything else. The number of 11-letter words we can obtain by the formula is given by the permuation of multisets formula (i.e. the multinomial coefficient) $\frac{11!}{1! 4! 4! 2!}$. Thus if we're given letter-counts like this as a representation of an obsevation, and do not know the actual observed _ordering_ in addition to letter counts, we have to count all possible orderings, of which there will be $\frac{11!}{1! 4! 4! 2!}$ possibilities. More generally, for a word-count vector $\boldsymbol{x} \in \mathbb{N}^{26}$ our probability will be

$$\text{Pr}(X = \boldsymbol{x}) = \frac{(\sum_l x_l)!}{\prod_l x_l!} \prod_l \phi_l ^{x_l}$$

which we recognise as the PMF of a multinomial distribution.

So the difference between the two PMFs -- the joint probability of an observed sequence of letters, or the probability of a all possible sequence of a letter mix -- depends only on whether or not the ordering is observed. Which choice you make has no effect on parameter estimation, since $\phi$ does not appear in the multinomial coefficient. It _will_ affect estimates of the probability of a particular observation given a parameter estimate $\hat{\phi}$. 

It does appear from this that if we assume letters in a word are exchangeable (or as a more pertinent example, words within a document are exchangeable), then the Multinomial is the only valid choice. Why this is not the case is FIXME

Well, the reality is that we _see_ the particular ordering. But what about `"The way the cat went"`. There's two `"the"`s there. Isn't that really _two_ sequences? Well, recall the binomial coefficient is a _combination_ and not a _permutation_. The multinomial coefficient similarly does... FIXME

### How this Impacts the Polya Distribution

This the marginalised Dirichlet-Multinomial distribution. See Wikipedia.

It inherits $\frac{n!}{x_i!}$ terms from the embedded multinomial. If we observe the sequence ordering, these disappear.


## Word Counts and Perplexity

Since the probability of a document will invariably diminish as the word count increases, methods have been proposed to adjust for this. One of the most common is perplexity. Given a model $\mathcal{M}$ trained on data $\mathcal{X}$ for which a posterior predictive distribution $p(x_*|\mathcal{X})$ exists, and a validation dataset $\mathcal{X}^*$ which -- due to an assumption of exchangeability -- decomposes into individual observations $x_d$, the perplexity is defined as 

$$
\text{Perp}(\mathcal{X}^*|\mathcal{X}) = \exp \left(
    - \frac{
        \sum_d \ln p(x_{d} | \mathcal{X})
    }{
        \sum_n N_d
    }
\right)
$$

$N_d$ is the count of tokens in $x_d$

FIXME More about it's relation to other metrics

# Document Completion

In practice topic models are often used to create latent representations of documents. The predictive likelihood doesn't measure the performance of the model for this task

Document completion is a metric which obtains a low-rank representation of a portion of a document $x_*^{(q)}$, and then evaluates the predictive likelihood of the remainder of the document $x_*^{(e)}$ given that fixed representation. The better the low-rank representation, the better it should explain the remaining words. Inspecting this metric therefore gives an assessment both of the model and model-inference procedure to obtain useful representations of documents.

For a mixture model, the documention completion metric involves first obtaining the posterior over topic assignment $z_*^{(q)}$ given $x_*^{(q)}$ and the model parameters. In the case of ML and MAP methods that can be obtained directly 

$$
p(z_*^{(q)} | x_*^{(q)}, \Phi) \propto p(x_*^{(q)}|\phi_k)p(z_*|\alpha)
$$

In the case of a Bayesian inference scheme, you need to evaluate

$$
\begin{align*}
p(z_*^{(q)}| x_*^{(q)}, \mathcal{X}) \propto \int_\Phi p(x_*^{(q)}|z, \Phi)p(z_*|\alpha)p(\Phi|\mathcal{X})
\end{align*}
$$
and re-normalise to obtain a categorical distribution. As with inference for the posterior-predictive, you can either use sampling, or approximate $q(\Phi) \approx p(\Phi|\mathcal{X})$ in such a way that the integral can be analytically evaluated. In the latter case, particularly with variational methods it is more straightforward to simply say

$$
\begin{align*}
\ln p(z_*^{(q)}| x_*^{(q)}, \mathcal{X}) \propto \mathbb{E}_{q(\Phi)}\left[\ln p(x_*^{(q)}, \Phi, z)\right]
\end{align*}
$$
which is of course the standard variational update equation with the variational posterior over $\Phi$ held fixed. Similarly, for inference using Gibbs sampling, one can sample from the posterior holding $\Phi$ fixed; or indeed draw samples from $\Phi$ on the assumption that in a "trained" model the chain will have converged so the posterior distribution over $\Phi$ is stationary.

Once obtained, we simply estimate the expected value of the posterior predictive distribution given these topic assignments. 

$$
\mathbb{E}_{z|x^{(q)}}\left[ p(x^{(e)}|z, \mathcal{X} \right] = \int_\Phi \sum_{z} p(x^{(e)}|z, \Phi)p(z|x^{(q)})p(\Phi|\mathcal{X})
$$

As before the three approaches to solving this are point-estimation, sampling, or deterministric distribution-approximation. In the sampling case we end up with

$$
\mathbb{E}_{z|x^{(q)}}\left[ p(x^{(e)}|z, \mathcal{X} \right] \approx \frac{1}{S} \sum_s \sum_k p(x^{(e)}|\phi^(s)_k)
$$
Where $z_s \sim p(z|x^{(q)})$ and $\hat{\Phi}_s \sim p(\Phi|\mathcal{X})$, or approximating distributions such as obtained via Gibbs sampling.

For cases where deterministic approximations of the distributions have been chosen for analytical tractability, such that $p(z|x^{(q)})p(\Phi|\mathcal{X}) = q(z)q(\Phi)$, i.e. variational methods, the approximation is


$$
\mathbb{E}_{z|x^{(q)}}\left[ p(x^{(e)}|z, \mathcal{X} \right] \approx \int_\Phi \sum_{z} p(x^{(e)}|z, \Phi)q(z|)q(\Phi|\mathcal{X})
$$

Typically document completion is translated into perplexity, to normalise for differing document lengths, and so it's the posterior log-likelihood that needs to be calculated. If we retain the use of approximate distributions we can see we're calcuating

$$
\begin{align}
 \ln \mathbb{E}_{q(z|x^{(q)})}\left[ \mathbb{E}_{q(\Phi)|
\mathcal{X})}\left[ p(x^{(e)}|z, \Phi) \right]\right]
\end{align}
$$

which, through application of Jensen's equality, will yield the variational lower bound. This simplifies the estimation of document-completion via perplexity.

# Case Studies

## Mixture of Multinomials - Expectation Maximisation

### Posterior Predictive

#### Point Estimate

For maximum likelihood, and MAP, we learn a _value_ $\hat{\Phi}$ (not a distribution) and then use that value, so we have

$$
\begin{align*}
p(x_*|\hat{\Phi}) = \sum_{z_*} p(x_*|z_*, \hat{\Phi})p(z|\alpha)
\end{align*}
$$

which for mixture models, simplifies to


$$
\begin{align*}
p(x_*|\Phi) = \sum_k p(x_*| \hat{\phi}_k)p(z=k|\alpha)
\end{align*}
$$

#### Expected

Does not apply in this case since we're not learning a distribution

### Document Completion

#### Point Estimate

The point estimate is simply

$$
\mathbb{E}_{p(z|x^{(e)}, \Phi)}\left[\ln p(x^{(e)}|z;\Phi)\right] = 
\sum_k p(x^{(e)}|\hat{\phi}_k)p(z=k|x^{(q)}; \hat{\Phi})
$$



#### Expected

Does not apply in this case since we're not learning a distribution over $\Phi$

## Mixture of Multinomials - Variational

### Posterior Predictive

#### Point Estimate

Selecting the point estimates $\hat{\phi} = \mathbb{E}_q[\phi]$ and $\hat{\alpha} = \mathbb{E}_q[\alpha]$ we obtain the following

$$
\begin{align*}
p(x|\mathcal{X}) 
    & = \int_z \int_\phi \int_\alpha p(x|z, \phi) p(z|\alpha) p(\alpha, \phi|\mathcal{X}) \\
    & \approx \int_z \int_\phi \int_\alpha p(x|z, \hat{\phi}) p(z|\hat{\alpha}) p(\alpha, \phi|\mathcal{X}) \\
    & = \int_z p(x|z, \hat{\phi}) p(z|\hat{\alpha}) \int_\phi \int_\alpha  p(\alpha, \phi|\mathcal{X}) \\
    & = \sum_k \hat{\alpha}_k \binom{\sum x_t}{x_1, \ldots, x_T} \prod_t \hat{\phi}_{kt}^{x_*t}
\end{align*}
$$
where the multinomial coefficient is given by $\binom{\sum x_t}{x_1, \ldots x_T} = \frac{(\sum x_t)!}{\prod_t x_t!}$. If we take the ordering as observed then this simplifies to

$$
p(x|\mathcal{X}) = \sum_k \hat{\alpha}_k \prod_t \hat{\phi}_{kt}^{x_*t}
$$


#### Expected

$$
p(x|\mathcal{X}) = \int_z \int_\phi \int_\alpha p(x|z, \phi) p(z|\alpha) p(\alpha, \phi|\mathcal{X})
$$

where we've already marginalized out $\mathcal{Z}$ from our parameter posterior $p(\alpha, \phi|\mathcal{X}) = \int_{\mathcal{Z}} p(\theta, \phi, \mathcal{Z}|\mathcal{X})$

If we use mean-field variational, then 

$$
p(\alpha, \phi, \mathcal{Z}|\mathcal{X}) \approx q(\alpha)q(\phi)q(\mathcal{Z})
$$
And so the marginalised parameter posterior is just $q(\alpha)q(\phi)$

And the posterior predictive is then

$$
\begin{align*}
p(x|\mathcal{X}) 
& \approx \int_z \int_\alpha \int_\phi p(x|z, \phi) p(z|\alpha)q(\alpha)q(\phi) 
\end{align*}
$$
We use a point estimate for the hyper-parameter $\alpha$ -- its mode $\hat{\alpha}$ -- and so, denoting the multinomial coefficient as $\binom{\sum x_t}{x_1, \ldots x_T}$; the Dirichlet normalizer as $H(\boldsymbol{\beta}) = \frac{\Gamma(\sum_t \beta_t)}{\prod_t \Gamma(\beta_t)}$ ; $\boldsymbol{\beta}$ as the parameter of the posterior over $\boldsymbol{\phi}$ , $q(\boldsymbol{\phi}; \boldsymbol{\beta})$; and finally by $x_{*t}$ the number of times that word $t$ occurred in the document $x_*$ we obtain

$$
\begin{align*}
p(x_*|\mathcal{X}) 
& \approx \sum_z \int_\Phi p(x_*|z, \Phi) p(z|\hat{\alpha}) q(\Phi) \\
& = \sum_k \int_\Phi p(x_*|\phi_k) \alpha_k q(\Phi) \\
& = \sum_k \int_{\phi_1} \ldots \int_{\phi_K} p(x_*|\phi_k) \alpha_k \prod_k q(\phi_k) \\
& = \sum_k \int_{\phi_1} \ldots \int_{\phi_K} 
\sum_k \left(\binom{\sum x_t}{x_1, \ldots x_T} \prod_t \phi_{kt}^{x_{*t}}\right) 
\alpha_k 
\left(
    \frac{1}{H(\boldsymbol{\beta}_k)} \prod_t \phi_{kt}^{(\beta_{kt} - 1)}
\right) \\
& = \sum_k \int_{\phi_1} \ldots \int_{\phi_K} 
\alpha_k 
\left(
    \frac{1}{H(\boldsymbol{\beta}_k)} \binom{\sum x_t}{x_1, \ldots x_T} \prod_t \phi_{kt}^{(x_{*t} - \beta_{kt} - 1)}
\right) \\
& = \sum_k
\alpha_k \binom{\sum x_t}{x_1, \ldots x_T}
\left(
    \frac{H(\boldsymbol{x_* + \beta_k})}{H(\boldsymbol{\beta}_k)}
\right)
\end{align*}
$$
which is a mixture of Polya distributions. Note in the third line that $q(\phi) = q(\phi_1), \ldots, q(\phi_K)$ (mean-field) and that $\mathbb{E}_Y[f(X)] = f(X)$ if $X$ is independent of $Y$. In this case $f(\phi_k)$ is independent of $\phi_j$ for all $j \neq k$ according to our factoried approximation.

If we take the ordering to be observed, then the multinomial coefficient falls out and we obtain.

$$
p(x_*|\mathcal{X}) = \sum_k
\alpha_k
\left(
    \frac{H(\boldsymbol{x_* + \beta_k})}{H(\boldsymbol{\beta}_k)}
\right)
$$

Note that for the selection of a topic we're drawing a 1-of-K indicator vector from a multinomial so the normalizer evaluates to 1. In this specific case the multinomial is equivalent to the categorical distribution.

In all the cases in this thesis, we will follow the prevailing convention in topic-model literature and take the ordering as observed, and therefore dropping the multinomial coefficient

In the case where a Bayesian update procedure is used for $\boldsymbol{\alpha}$ with an approximate variational procedure $q(\boldsymbol{\alpha}; \boldsymbol{u})$ having a Dirichlet distribution, we obtain the following predictive posterior:

$$
p(x_*|\mathcal{X}) = \sum_k
    \frac{H(\delta_k + \boldsymbol{u})}{H(\boldsymbol{u})}
    \frac{H(\boldsymbol{x_* + \beta_k})}{H(\boldsymbol{\beta}_k)}
$$
where $\delta_k$ is a 1-of-K indictor vector in which position $k$ is set to 1 and all other positions are set to 0.


### Document Completion

In order to report document-completion using perplexity we need an estimate of 

$$
\ln \mathbb{E}_{z|x^{(q)}}\left[ \mathbb{E}_{\Phi|\mathcal{X}}[ p(x^{(e)}|z, \Phi) ] \right]
$$

#### Point Estimate

As with the point estimate of log likelihood, this is just a simple substitution

$$
\ln \mathbb{E}_{z|x^{(q)}}\left[ \mathbb{E}_{\Phi|\mathcal{X}}[ \ln p(x^{(e)}|z, \Phi) ] \right]
\approx
\ln \left(\sum_k p(x_* | \hat{\phi}) p(z=k|x^{(q)})\right)
$$ 

This is the familiar log-sum-exp function. In this case a problem is that due to document sizes, probabilities may collapse to zero before we calculate log. However using the identity

$$
\begin{align*}
\ln \left(\sum_k \exp(l_k)\right) & = \ln \left(\sum_k \exp(l_k - a + a)\right) \\
  & = \ln \left(\sum_k \exp(l_k - a)\exp(a)\right) \\
  & = \ln \left(\exp(a)\sum_k \exp(l_k - a) \right)\\
  & = a + \ln \left(\sum_k \exp(l_k - a)\right)
\end{align*}
$$
we can evaluate the log-probability $l_k = \ln p(x_* | \hat{\phi}) p(z=k|x^{(q)})$ for each topic and set $a = \max_k l_k$ ensures that the majority of terms should not catastrophically underflow and none should overflow. This identity is used in all cases where we have to take the log of a probability over words. 

In the next chapter we will look at cases where a log-sum-exp function occurs in the full log-joint probability and we need to look at other bounds in order to perform parameter infernce.

#### Expected

In this case we use the variational posteriors $q(\boldsymbol{z}; \boldsymbol{r}) \approx p(z|x^{(q)}, \mathcal{X})$ and $q(\Phi; \boldsymbol{\beta}_1, \ldots, \boldsymbol{\beta}_K) \approx p(\Phi|\mathcal{X})$

$$
\begin{align*}
\ln \mathbb{E}_{z|x^{(q)}, \mathcal{X}}\left[ \mathbb{E}_{\Phi|\mathcal{X}}[ p(x^{(e)}|z, \Phi) ] \right] 
    & \approx \ln \mathbb{E}_{q(z)}\left[ \mathbb{E}_{q(\Phi)}[ p(x^{(e)}|z, \Phi) ] \right] \\
    & \geq \ln \mathbb{E}_{q(z)}\left[ \mathbb{E}_{q(\Phi)}[ \ln p(x^{(e)}|z, \Phi) ] \right]
\end{align*}
$$

in which we use Jensen's bound to obtain the last term, which is the variational lower bound. Assuming  an appropriate choice of approximate posteriors, it is possible to write this out analytically:

$$
\begin{align*}
\ln \mathbb{E}_{z|x^{(q)}, \mathcal{X}}\left[ \mathbb{E}_{\Phi|\mathcal{X}}[ p(x^{(e)}|z, \Phi) ] \right]
 & \geq
 \sum_k \sum_t \mathbb{E}[z_k] x^{(e)}_t \mathbb{E}[\ln \phi_{kt}]
 + \sum_k -\ln H(\boldsymbol{\beta}_k) +  \sum_t (\beta_t - 1) \mathbb{E}[\ln \phi_{kt}]
\end{align*}
$$

where $\mathbb{E}[\ln \phi_{kt}] = \Psi(\beta_{kt}) - \Psi(\sum_v \beta_{kv})$, and $\Psi(\cdot)$ indicates the digamma function.

It is conspicuous that we don't need to use the log-sum-exp trick in this case: this is due to the use of Jensen's inequality to put the log _inside_ the expecation, i.e. the sum over topic-choices. In the case where the distributions are peaky, have low variance, and so are well-approximated by point methods, the point estimate may consequently be a more accurate estimate of the posterior probability than the variational lower-bound.

CF James Hensman

## Mixture of Multinomials - Gibbs

### Posterior Predictive

#### Point Estimate

Selecting the point estimates $\hat{\Phi} = \frac{1}{S}\sum_s \Phi_s$ and $\hat{\alpha} = \frac{1}{S}\sum_s \alpha_s$ for $\alpha_s, \Phi_s \sim p(\alpha, \Phi|\mathcal{X})$ we obtain the same estimate as 

$$
p(x|\mathcal{X}) = \sum_k \hat{\alpha}_k \prod_t \hat{\phi}_{kt}^{x_*t}
$$

where we've dropped the multinomial coefficient on the assumption that the ordering is observed.

#### Expected

$$
\begin{align*}
p(x|\mathcal{X}) 
    & = \int_z \int_\phi \int_\alpha p(x|z, \phi) p(z|\alpha) p(\alpha, \phi|\mathcal{X}) \\
    & \approx \frac{1}{S} \sum_s \sum_k \alpha^{(s)}_{k} \prod_t \phi_{kt}^{x_{*t}}
\end{align*}
$$
where $\alpha_s, \Phi_s \sim p(z, \alpha, \Phi | x, \mathcal{X})$. In this instance $z_s$ is not a K-dimensional indicator vector, it is a scalar indicating the chosen topic. 

### Document Completion

#### Point Estimate

As with the point estimate of posterior predictive, this is just a simple substitution

$$
\begin{align*}
\ln \mathbb{E}_{z|x^{(q)}, \mathcal{X}}\left[ \mathbb{E}_{\Phi|\mathcal{X}}[ \ln p(x^{(e)}|z, \Phi) ] \right]
& \approx
\ln \left(\sum_k p(x_* | \hat{\phi}) p(z=k|x^{(q)})\right) 
\end{align*}
$$ 

where $\hat{\Phi} = \frac{1}{S}\sum_s \Phi_s$ and $\hat{\alpha} = \frac{1}{S}\sum_s \alpha_s$ for $\alpha_s, \Phi_s \sim p(\alpha, \Phi|\mathcal{X})$ . We use the same log-sum-exp trick again to avoid underflow.

The posterior distribution $p(z|x^{(q)})$ can be inferred in a few different ways. In the most simple way, using point-estimates $\hat{\Phi}$ and $\hat{\alpha}$ one can analytically evaluate it as for the Mixture of Multinomials with EM.

A second way is to sample from $p(z, \mathcal{Z}, \boldsymbol{\alpha}, \Phi| x^{(q)}, \mathcal{X})$, (e.g. via Gibbs sampling), and then obtain $p(z|x^{(q)}, \mathcal{X}) = \int_\mathcal{Z} \int_\boldsymbol{\alpha} \int_\Phi p(z, \mathcal{Z}, \boldsymbol{\alpha}, \Phi| x^{(q)}, \mathcal{X}) \approx \frac{1}{S} \sum_s z_s$. To obtain the samples one can presume that the posterior distributions over $\Phi$ and $\boldsymbol{\alpha}$ have converged, and so being stationary, will not alter if one draws further samples from them. This also relies on the assumption that $x$ is from the sample distribution as $\mathcal{X}$ and so -- in the practical finite world of computation -- will not peturb those distributions.

The third approach is to retain $S$ samples from the converged, stationary chain during the "training" phase, and then re-use them to draw $S$ additional samples of $z$. 

In our work we will exclusvely use the second approach.

#### Expected

As in the point-estimate method, we use the augmented dataset approach to draw samples from $p(z|x, \mathcal{X})$. The the expected document completion metric is 

$$
\begin{align*}
\ln \mathbb{E}_{z|x^{(q)}, \mathcal{X}}\left[ \mathbb{E}_{\Phi|\mathcal{X}}[ p(x^{(e)}|z, \Phi) ] \right] 
    & \approx \ln \mathbb{E}_{q(z)}\left[ \mathbb{E}_{q(\Phi)}[ p(x^{(e)}|z, \Phi) ] \right] \\
    & \geq \ln \mathbb{E}_{q(z)}\left[ \mathbb{E}_{q(\Phi)}[ \ln p(x^{(e)}|z, \Phi) ] \right]
\end{align*}
$$


## LDA - Variational

### Posterior Predictive

#### Point Estimate

In this case we have $\hat{\alpha} = \mathbb{E}\left[ \alpha \right]$ and $\hat{\Phi} = \mathbb{E}\left[\Phi\right]$

$$
\begin{align*}
p(x|\mathcal{X}) & = \int_\alpha \int_\Phi \int_\theta \sum_z 
p(x|z,\Phi)p(z|\theta)p(\theta|\alpha)p(\alpha, \Phi|\mathcal{X}) \\
& \approx \int_\alpha \int_\Phi \int_\theta \sum_z p(x|z,\hat{\Phi})p(z|\theta)p(\theta|\hat{\alpha})p(\alpha, \Phi|\mathcal{X}) \\
& \approx \int_\theta \sum_z p(x|z,\hat{\Phi})p(z|\theta)p(\theta|\hat{\alpha})\int_\alpha \int_\Phi p(\alpha, \Phi|\mathcal{X}) \\
& = \int_\theta \sum_z 
p(x|z,\hat{\Phi})p(z|\theta)p(\theta|\hat{\alpha})
\end{align*}
$$

We can analytically solve the integral over $\theta$, obtaining the Polya, or Dirichlet-Multinomial distribution, parameterised by the number of draws (i.e. terms in $x$) and $\alpha$. Since we presume the ordering is observed, we drop the multinomial coefficient and so obtain:

$$
\begin{align*}
p(x|\mathcal{X}) 
    & \approx \sum_{z} \frac{H(\sum_n \boldsymbol{z}_{n} + \alpha)}{H(\alpha)} \prod_n \prod_k p(x_n|\hat{\phi}_k)^{z_{nk}} \\
    & = \sum_{z} \frac{H(\sum_n \boldsymbol{z}_{n} + \alpha)}{H(\alpha)} \prod_n \prod_t \prod_k \hat{\phi}_{kt}^{x_{nt}z_{nk}}
\end{align*}
$$

where $z_{n}$ is a 1-of-$K$ vector indicating which of topic was chosen. One element that is clearly evident from this is that the choice of $z_{n}$ for each term is solely determined by the _prior_ $\boldsymbol{\alpha}$

With N terms, each with K choices, we have to sum over $K^N$ possible values of $z$, which is not feasible for all but the toy examples. Similarly, unless the prior is extraordinarily peaked, drawing $S << K^N$ samples from $p(z_1, \ldots z_n|\alpha) = \frac{H(\sum_n \boldsymbol{z}_{n} + \alpha)}{H(\alpha)}$ to estimate $\mathbb{E}_{z_1, \ldots z_n|\alpha}\left[ \prod_n \prod_k p(x_n|\phi_k)^{z_{nk}} \right]$ will not give good estimates.


While topic-models provide better latitude in the inference of components $\phi_1, \ldots, \phi_K$, it's clear from this formalism that the predictive likelihood might not differ very much from that for a mixture of multinomials. In practice, it s the document-completion metric which gives a better evaluation of these models' abilities to infer latent representations


##### Inferred $\theta$ approach

Ultimately, the best path to an approximation is to continue with point-estimates. Given that we've established that an _exact_ solution to $\int_\theta \sum_z p(x|z, \Phi)p(z|\theta)p(\theta|\hat{\alpha})$ is entirely determined by $\hat{\alpha}$, it makes sense to use the point approximation $\hat{\theta} = \mathbb{E}_{\theta|\hat{\alpha}}\left[ \theta \right] = \left(\frac{\hat{\alpha}_k}{\sum_j \hat{\alpha}_j}\right)_{k=1}^K$. Plugging this in, we get

$$
\begin{align*}
p(x | \mathcal{X}) 
& \approx p(x | \mathcal{X}; \hat{\Phi}, \hat{\alpha}) \\
& = \int_\theta \sum_z 
p(x|z,\hat{\Phi})p(z|\theta)p(\theta|\hat{\alpha}) \\
& \approx \sum_z 
p(x|z,\hat{\Phi})p(z|\hat{\theta})\int_\theta p(\theta|\hat{\alpha}) \\
& = \sum_z p(x|z,\Phi)p(z|\hat{\theta}) \\
& = \prod_n \prod_t \sum_k \left(\hat{\phi}_{kt} \hat{\theta}_k\right)^{x_{nt}}
\end{align*}
$$

This just involves a sum over $K\cdot N$ different settings of $z$

#### Expected

As has been explained in the previous section, even if you take point-estimates of $\Phi$ and $\alpha$, there is still no tractable solution to obtaining the expected posterior probablity of an observation with LDA. Were we to use importance sampling our estimates could have infinite variance since the variational proposal distributions are likely to underestimate the tails compared to the true distribution

One can substitute the variational posteriors for the true posterior, and then analytically integrate out $\phi_1, \ldots, \phi_K$ completely, as with $\theta$. In this case we denote the parameters of the variational posteriors as $\alpha^{(N)}$ and $\beta^{(N)}$

$$
\begin{align*}
p(x|\mathcal{X}) 
& = \int_\theta \sum_z \int_{\Phi} p(x|z, \Phi)p(z|\theta)p(\theta|\alpha)p(\alpha, \Phi|\mathcal{X}) \\
& \approx \int_\theta \sum_z \int_{\phi_1} \ldots \int_{\phi_K} p(x|z, \Phi)p(z|\theta)p(\theta|\alpha)q(\alpha)\prod_k q(\phi_k) \\
& = \int_\theta \sum_z \int_{\phi_1} \ldots \int_{\phi_K} 
\prod_n \prod_k \prod_t (\phi_{kt} \theta_k)^{z_{nk}x_{nt}}\frac{1}{H(\alpha^{(N)})}
\prod_k \theta_k^{\alpha^{(N)}_k - 1}\frac{1}{H(\beta^{(N)})}\prod_t \phi_{kt}^{\beta^{(N)}_t - 1} \\
& = \int_\theta \sum_z \int_{\phi_1} \ldots \int_{\phi_K} 
\prod_k \prod_t (\phi_{kt} \theta_k)^{\sum_n z_{nk}x_{nt}}\frac{1}{H(\alpha^{(N)})}
\prod_k \theta_k^{\alpha^{(N)}_k - 1}\frac{1}{H(\beta^{(N)})}\prod_t \phi_{kt}^{\beta^{(N)}_t - 1} \\ 
& = \int_\theta \sum_z \int_{\phi_1} \ldots \int_{\phi_K} 
\frac{1}{H(\alpha^{(N)})}\prod_k \theta_k^{\sum_n \sum_t x_{nt}z_{nk}} \prod_k \frac{1}{H(\beta^{(N)_k})}\prod_t \phi_{kt}^{\sum_n z_{nk}x_{nt} + \beta^{(N)}_t - 1} \\
& = \sum_z  
\frac{H(\sum_n \sum_t x_{nt}\boldsymbol{z}_{n} + \alpha^{(N)})}{H(\alpha^{(N)})} \prod_k \frac{H(\sum_n z_{nk}x_{n} + \beta^{(N)}_k)}{H(\beta^{(N)}_k)} \\
\end{align*}
$$

however this then leads to the same issue of having to sum over the $K^N$ possible settings of $z$; which -- given that the prior is not likely to be peaked -- cannot be well-approximated by sampling $S <<  K^N$ samples.

In general, practitioners resort to point estimates, using the means  --  according to the variational posterior distributions -- of the parameters of interest.

The only other option that encapsulates the uncertainty around these parameter estimates is to employ the variational lower bound as an appoximation, but this is by construction lower than the true posterior log-probability.

### Document Completion

#### Point Estimate

As with the point estimate of posterior predictive, this a simple substitution, using $\hat{\theta} = \mathbb{E}_{\theta|x^{(q)}, \mathcal{X}}\left[\theta\right] \approx \mathbb{E}_q\left[\theta\right] = \frac{\alpha^{(N)}_k}{\sum_j \alpha^{(N)}_j}$ i.e. the expected value according to our variational approximation to the true posterior.

$$
\begin{align*}
\mathbb{E}_{\theta|x^{(q)}, \mathcal{X}}\left[ \mathbb{E}_{\Phi|\mathcal{X}}[ \ln \sum_k p(x^{(e)}|z, \Phi)p(z|\theta) ] \right]
& \approx \ln \left(\sum_k p(x_* | \hat{\phi}) p(z=k|\hat{\theta}) \right) \\
& \approx \ln \left(\sum_k p(x_* | \hat{\phi}) \theta_k \right) \\
& = \ln \left(\sum_k \prod_n \prod_t \left(\phi_{kt} \theta_k \right)^{x^*_{nt}} \right)
\end{align*}
$$ 

where additionally $\hat{\Phi} = \mathbb{E}_q[\Phi]$. We use the same log-sum-exp trick again to avoid underflow. 

In order to obtain a variational posterior, we run the variational inference procedure, holding the estimates of all other random variables fixed except for $\theta$.

#### Expected

In this case we use the variational posteriors $q(z)q(\boldsymbol{\theta}; \alpha^{(N)})q(\Phi; \boldsymbol{\beta}^{(N)}_1, \ldots, \boldsymbol{\beta}^{(N)}_K) \approx p(\theta,z, \Phi|x^{(q)}, \mathcal{X})$ 

$$
\begin{align*}
\ln \mathbb{E}_{\theta, \Phi, z|x^{(q)}, \mathcal{X}}\left[ p(x^{(e)}, z, \theta, \Phi) ] \right] 
    & \approx \ln \mathbb{E}_{q(\theta)}\left[ \mathbb{E}_{q(\Phi)}\left[ \mathbb{E}_{q(z)}\left[p(x^{(e)}, z, \theta, \Phi) \right]  \right] \right] \\
    & \geq \mathbb{E}_{q(\theta)}\left[ \mathbb{E}_{q(\Phi)}\left[ \mathbb{E}_{q(z)}\left[\ln p(x^{(e)}, z, \theta, \Phi) \right]  \right] \right]
\end{align*}
$$

in which we use Jensen's bound to obtain the last term, which is the variational lower bound. Assuming  an appropriate choice of approximate posteriors, it is possible to write this out analytically:

$$
\begin{align*}
\ln \mathbb{E}_{\theta, \Phi, z|x^{(q)}, \mathcal{X}}\left[ p(x^{(e)}|z, \theta, \Phi) ] \right] 
 & \geq \sum_n \sum_k \sum_t \mathbb{E}[z_{nk}] x^{(e)}_{nt} \mathbb{E}[\ln \phi_{kt}] \\
 & + \sum_n \sum_k \mathbb{E}[z_{nk}]\mathbb{E}[\ln \theta_{k}] \\
 & -\ln H(\boldsymbol{\alpha}^{(N)}_k) +  \sum_k (\alpha_k - 1) \mathbb{E}[\ln \theta_{k}]\\
 & + \sum_k -\ln H(\boldsymbol{\beta}^{(N)}_k) +  \sum_t (\beta_t - 1) \mathbb{E}[\ln \phi_{kt}]
\end{align*}
$$

where $\mathbb{E}[\ln \phi_{kt}] = \Psi(\beta_{kt}) - \Psi(\sum_v \beta_{kv})$, and $\Psi(\cdot)$ indicates the digamma function; and similarly for $\theta$ which also has a Dirichlet distribution

As with the mixture of multinomials, we don't need to employ the log-sum-exp trick in this case due to Jensen's inequality putting the log _inside_ the expectation.

## LDA - Collapsed Variational

### Posterior Predictive

#### Point Estimate

In the case of LDA/CVB one can derive the expected values of the marginalized random variables: $\hat{\theta}_d = \mathbb{E}[\theta_{dk}] = \frac{\alpha + \mathbb{E}_q[n_{dk\cdot}]}{\sum_k \alpha_k + \mathbb{E}_q[n_{d \cdot \cdot}]}$ and $\hat{\phi}_{kt} = \mathbb{E}[\phi_{kt}] = \frac{\beta + \mathbb{E}_q[n_{\cdot k t}]}{\sum_t \beta_t + \mathbb{E}_q[n_{\cdot k \cdot}]}$, where we use the Gaussian approximation for the sums of Bernoulli trials $n_{dkt}$ in order to obtain a simple form for the expectation

In the case of the predictive distribution we just end up with the same equation as for LDA/VB, with $\hat{\theta} = \mathbb{E}_{\theta|\hat{\alpha}}\left[\theta\right] = \left(\frac{\alpha_k}{\sum_j \alpha_j}\right)_{k=1}^K$

$$
\begin{align*}
p(x | \mathcal{X}) 
& \approx p(x | \mathcal{X}; \hat{\Phi}, \hat{\alpha}) \\
& = \int_\theta \sum_z 
p(x|z,\hat{\Phi})p(z|\theta)p(\theta|\hat{\alpha}) \\
& \approx \sum_z 
p(x|z,\hat{\Phi})p(z|\hat{\theta})\int_\theta p(\theta|\hat{\alpha}) \\
& = \sum_z p(x|z,\Phi)p(z|\hat{\theta}) \\
& = \prod_n \prod_t \sum_k \left(\hat{\phi}_{kt} \hat{\theta}_k\right)^{x_{nt}}
\end{align*}
$$

This is the method given in the actual paper, except that they used $\hat{\theta} = \mathbb{E}_q[\theta] \approx \mathbb{E}_{\theta|x, \mathcal{X}}[\theta]$ which is of course invalid

#### Expected

The variational bound for LDA/CVB is expressed entirely in terms of the term-topic assignments $\boldsymbol{z}_n$. 

The appropriate likelihood distribution is given by the marginalised equation (the product of Polyas) with $K^N$ summations over all possible values $z$. However this is intractable.

Ultimately, there is no tractable solution to this.

### Document Completion

#### Point Estimate

By running the inference scheme holding $n_{\cdot k t}$ fixed, and $n_{d k \cdot}$ fixed for all $d$ in the training set, one can obtain a set of term-level topic-assignments for the new document, and thereby use the pre-existing equations to get the mean values of $\hat{\theta}$ and $\hat{\Phi}$. Thes can then be substituted as for LDA to get the point estimate of the document completion. 

#### Expected

> The thing that makes this so hard is that you don't have the global parameters stored explicity. You can only proceed by working backwards. But that that point you're in point-estimate land again.
> Perhaps you could just use the training and new results to get the variational bound for the augmented data, then take away the variational bound for the training data alone. 



## LDA - Gibbs

### Posterior Predictive

#### Point Estimate

#### Expected

### Document Completion

#### Point Estimate

#### Expected

## HDP - Variational

### Posterior Predictive

#### Point Estimate

#### Expected

### Document Completion

#### Point Estimate

#### Expected

## LDA - Variational

### Posterior Predictive

#### Point Estimate

#### Expected

### Document Completion

#### Point Estimate

#### Expected

# How does this compare with other methods?

## Heinrich

He used the point-estimate substitution trick:

$$
\begin{align*}
p(x_*|\mathcal{M}) 
& = \prod_n \sum_k p(x_{*n}|\phi_k)p(z_{*n} = k|\theta_*) \\
& = \prod_t \left( \sum_k \phi_{kt} \theta_{*k} \right)^{n_{*t}}
\end{align*}
$$

So why does this work then?

## Teh

Used point estimates the same as Heinrich

## Wallach 

It's a really funny paper. Given a fixed, point-estimate for $\Phi$ and $\alpha$, she's concerned with estimating

$$
p(x|\Phi, \alpha)
$$

which is just a component of the broader method

$$
p(x|\mathcal{X}) = \int_\Phi \int_\alpha p(x|\Phi, \alpha)p(\Phi, \alpha|\mathcal{X})
$$

It's possible to obtain samples via Gibb's sampling

$$
z_{*nk} \propto \phi_{kw_{*n}} \frac{
    n_{*k}^{(\setminus n)} + \alpha_k
}{
    n - 1 + \sum_j \alpha_j
}
$$

And of course, once that's done, it's possible to estimate a sample of $\theta_{*}$ as:

$$
\theta_*^{(s)} = \frac{
    n_{*k}^{(s)} + \alpha_k
}{
    n - 1 + \sum_j \alpha_j
}
$$

And so determine the likelihood just as the Heinrich and Teh  (section "5.1 Estimated $\theta$" method)

$$
\begin{align}
p(x|\Phi, \alpha) = \frac{1}{S}\sum_s 
\left(
    \prod_t \left(
        \sum_k \phi_kt \theta^{(s)}_{*k}
    \right)^{n_{*t}} 
  \right)
\end{align}
$$

So the aim of the paper is how to estimate the embedded document specific $\theta_*$ (and so $z_{*n}$) for all 

<font color='red'>
    Something that this illustrates is that neither Teh nor Heinrich are giving the proper posterior probability $p(x|\mathcal{X})$ in the context of document-completion: i.e. instead of reporting $p(x^{(e)}|x^{(q)}, \mathcal{X})$ they are instead reporting $p(x^{(e)}|x^{(q)}, \Phi, \alpha)$
</font>

Another thing that follows is that you can't really use the variational bound as a proxy for predictive likelihood. The variational bound is an approximation of the marginal, likelihood according to prior distributions, but a different bound arises when doing posterior predictive, as shown by Bishop with the Variational Mixture of Gaussians example.

Another issue is that she goes straight to sampling without every looking at analytical solutions. $p(z|\alpha)$ has a Polya distribution (since $\theta_{*}$ got marginalized out. 

After that though it's the usual issue. She's dissatisfied with the decomposition

$$
p(w|\Phi, \alpha) = \int_\theta \sum_z p(w|\Phi, z)p(z|\theta)p(\theta|\alpha)
$$

as it essentially means sampling from the prior. However by analogy with mixture models, this is in fact the proper way to go about things. All other methods to avoid this are a bit squiffy.

> Importance sampling does not work well when sampling from high-dimensinal disributions. Unless the proposal distribution is a near-perfect approxiamtion to the target distribution, the variance of the estimator will be very large. When sampling continuous values, such as $\theta$, the estimator may have infinite variance.

The harmonic mean method is probably most justifiable, if it still a little squiffy.

What makes this all justifiable -- ish -- is that in documention completion you are using one set of words to sample a posterior which you then use to evaluate the remaining set of words.